In [1]:
import pandas as pd
import numpy as np

In [2]:
# Task 1: Creating DataFrames
employee_data = {
    "ID": ["A001", "A002", "A003", "A004", "A005"],
    "Name": ["John Alter", "Alice Luxumberg", "Tom Sabestine", "Nina Adgra", "Amy Johny"],
    "Gender": ["M", "F", "M", "F", "F"],
    "City": ["Paris", "London", "Berlin", "Newyork", "Madrid"],
    "Age": [25, 27, 29, 31, 30]
}
employee_df = pd.DataFrame(employee_data)

In [3]:
seniority_data = {
    "ID": ["A001", "A002", "A003", "A004", "A005"],
    "Designation Level": [2, 2, 3, 2, 3]
}
seniority_df = pd.DataFrame(seniority_data)

In [4]:
project_data = {
    "ID": ["A001", "A002", "A003", "A004", "A005", "A002", "A005", "A003", "A001", "A003", "A001", "A004", "A004", "A005"],
    "Project": ["Project 1", "Project 2", "Project 3", "Project 4", "Project 5", "Project 6", "Project 7", "Project 8", "Project 9", "Project 10", "Project 11", "Project 12", "Project 13", "Project 14"],
    "Cost": [1002000, 2000000, 4500000, 5500000, np.nan, 680000, 400000, 350000, np.nan, 300000, 2000000, 1000000, 3000000, 200000],
    "Status": ["Finished", "Ongoing", "Finished", "Ongoing", "Finished", "Failed", "Finished", "Failed", "Ongoing", "Finished", "Failed", "Ongoing", "Finished", "Finished"]
}
project_df = pd.DataFrame(project_data)

In [5]:
# Saving CSV Files
employee_df.to_csv("Employee.csv", index=False)
seniority_df.to_csv("Seniority.csv", index=False)
project_df.to_csv("Project.csv", index=False)

In [6]:
# Task 2: Handling Missing Values with Running Average
costs = project_df["Cost"].copy()
for i in range(len(costs)):
    if pd.isna(costs[i]):
        costs[i] = np.nanmean(costs[:i])  # Running average
project_df["Cost"] = costs

In [8]:
# Task 3: Splitting Name Column
employee_df[['First Name', 'Last Name']] = employee_df['Name'].str.split(n=1, expand=True)
employee_df.drop(columns=['Name'], inplace=True)

In [9]:
# Task 4: Merging DataFrames
final_df = employee_df.merge(seniority_df, on="ID").merge(project_df, on="ID")

In [10]:
# Task 5: Adding Bonus Column (5% for Finished Projects)
final_df['Bonus'] = np.where(final_df['Status'] == 'Finished', final_df['Cost'] * 0.05, 0)

In [11]:
# Task 6: Demoting Employees with Failed Projects
failed_projects = final_df[final_df['Status'] == 'Failed']["ID"].unique()
final_df.loc[final_df['ID'].isin(failed_projects), 'Designation Level'] += 1
final_df = final_df[final_df['Designation Level'] <= 4]  # Remove employees with Designation Level > 4

In [12]:
# Task 7: Adding Titles & Removing Gender Column
final_df['First Name'] = np.where(final_df['Gender'] == 'M', "Mr. " + final_df['First Name'], "Mrs. " + final_df['First Name'])
final_df.drop(columns=['Gender'], inplace=True)

In [14]:
# Task 8: Promoting Employees Over Age 29
final_df.loc[final_df['Age'] > 29, 'Designation Level'] -= 1

In [15]:
# Task 9: Summing Project Costs per Employee
total_cost_df = final_df.groupby(['ID', 'First Name'])['Cost'].sum().reset_index()
total_cost_df.rename(columns={'Cost': 'Total Cost'}, inplace=True)

In [16]:
# Task 10: Employees in Cities Containing 'o'
filtered_employees = final_df[final_df['City'].str.contains('o', case=False)]

In [17]:
# Displaying Outputs
print("Final DataFrame:")
print(final_df.head())

Final DataFrame:
     ID    City  Age  First Name  Last Name  Designation Level     Project  \
0  A001   Paris   25    Mr. John      Alter                  3   Project 1   
1  A001   Paris   25    Mr. John      Alter                  3   Project 9   
2  A001   Paris   25    Mr. John      Alter                  3  Project 11   
3  A002  London   27  Mrs. Alice  Luxumberg                  3   Project 2   
4  A002  London   27  Mrs. Alice  Luxumberg                  3   Project 6   

        Cost    Status    Bonus  
0  1002000.0  Finished  50100.0  
1  2210312.5   Ongoing      0.0  
2  2000000.0    Failed      0.0  
3  2000000.0   Ongoing      0.0  
4   680000.0    Failed      0.0  


In [18]:
print("\nTotal Project Cost DataFrame:")
print(total_cost_df)


Total Project Cost DataFrame:
     ID  First Name  Total Cost
0  A001    Mr. John   5212312.5
1  A002  Mrs. Alice   2680000.0
2  A003     Mr. Tom   5150000.0
3  A004   Mrs. Nina   9500000.0
4  A005    Mrs. Amy   3850500.0


In [19]:
print("\nEmployees from Cities with 'o':")
print(filtered_employees[['ID', 'First Name', 'City']])


Employees from Cities with 'o':
      ID  First Name     City
3   A002  Mrs. Alice   London
4   A002  Mrs. Alice   London
8   A004   Mrs. Nina  Newyork
9   A004   Mrs. Nina  Newyork
10  A004   Mrs. Nina  Newyork
